# A Neural Algorithm of Arctic Style 画風変換
# Tensor Flowで実装する

# 長文説明バージョン

In [1]:
import tensorflow as tf
import numpy as np
import scipy.io
import scipy.misc
import os

VGGの学習済みの重みを利用します。その為、MATLABのmat形式で配布されているファイルをダウンロードします。
URL: http://www.vlfeat.org/matconvnet/pretrained/  
(VGG-VDモデルの、imagenet-vgg-verydeep-19.matをダウンロードします。)  
今回は、ダウンロードしたファイルを  
`[カレントディレクトリ]>[models]`  
に保存しています。

In [2]:
VGG_MODEL = "models/imagenet-vgg-verydeep-19.mat"

画風変換に利用する画像を設定していきます。  
スタイル画像の特徴をコンテンツ画像に適用していき、その結果が生成画像として出力されます。  
また、生成画像は最適化回数ごとに出力されますので、ディレクトリを指定します。  

In [3]:
CONTENT_IMG = 'images/SetoBridge.jpg'  # コンテンツ画像
STYLE_IMG = 'images/StarryNight.jpg'  # スタイル画像

OUTPUT_DIR = 'results'  # 生成画像ディレクトリ
OUTPUT_IMG = 'result.png'  # 生成画像ファイル

VGGは600×300×3の画像を想定しているので、それぞれのサイズを設定する。  
平均値をゼロにするため、VGGの訓練データの平均画素値[123.68, 116.779, 103.939]を引かなければならない。(その為、平均画素値はVGGモデルにより異なる。)　　


In [4]:
IMAGE_W = 800
IMAGE_H = 500
#  入力画像から平均画素値を引くための定数(reshapeでそのまま引けるようにする)
MEAN_VALUES = np.array([123, 117, 104]).reshape((1,1,1,3))

## ネットワークの定義

In [5]:
def build_net(ntype, nin, rwb=None):
    """
    ネットワークの各層をTensorFlowで定義する関数
    : param ntype: ネットワークの層のタイプ(ここでは、畳み込み層もしくは、プーリング層)
    : param nin: 前の層
    : param rwb: VGGの最適化された値
    """
    if ntype == 'conv':
        return tf.nn.relu(tf.nn.conv2d(nin, rwb[0], strides=[1, 1, 1, 1], padding='SAME') + rwb[1])
    elif ntype == 'pool':
        return tf.nn.avg_pool(nin, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [6]:
def get_weight_bias(vgg_layers, i):
    """
    VGGの各層の最適化された重みとバイアスを取得する関数
    : param vgg_layers: ネットワークの層
    : param i:
    """
    weights = vgg_layers[i][0][0][2][0][0]
    weights = tf.constant(weights)
    bias = vgg_layers[i][0][0][2][0][1]
    bias = tf.constant(np.reshape(bias, (bias.size)))
    return weights, bias

In [7]:
def build_vgg19(path):
    """
    TensorFlowでVGGネットワークを構成する関数
    : param path: VGGの学習済みモデルのファイルのパス
    """
    net = {}
    vgg_rawnet = scipy.io.loadmat(path)
    vgg_layers = vgg_rawnet['layers'][0]
    net['input'] = tf.Variable(np.zeros((1, IMAGE_H, IMAGE_W, 3)).astype('float32'))
    net['conv1_1'] = build_net('conv',net['input'],get_weight_bias(vgg_layers,0))
    net['conv1_2'] = build_net('conv',net['conv1_1'],get_weight_bias(vgg_layers,2))
    net['pool1']   = build_net('pool',net['conv1_2'])
    net['conv2_1'] = build_net('conv',net['pool1'],get_weight_bias(vgg_layers,5))
    net['conv2_2'] = build_net('conv',net['conv2_1'],get_weight_bias(vgg_layers,7))
    net['pool2']   = build_net('pool',net['conv2_2'])
    net['conv3_1'] = build_net('conv',net['pool2'],get_weight_bias(vgg_layers,10))
    net['conv3_2'] = build_net('conv',net['conv3_1'],get_weight_bias(vgg_layers,12))
    net['conv3_3'] = build_net('conv',net['conv3_2'],get_weight_bias(vgg_layers,14))
    net['conv3_4'] = build_net('conv',net['conv3_3'],get_weight_bias(vgg_layers,16))
    net['pool3']   = build_net('pool',net['conv3_4'])
    net['conv4_1'] = build_net('conv',net['pool3'],get_weight_bias(vgg_layers,19))
    net['conv4_2'] = build_net('conv',net['conv4_1'],get_weight_bias(vgg_layers,21))
    net['conv4_3'] = build_net('conv',net['conv4_2'],get_weight_bias(vgg_layers,23))
    net['conv4_4'] = build_net('conv',net['conv4_3'],get_weight_bias(vgg_layers,25))
    net['pool4']   = build_net('pool',net['conv4_4'])
    net['conv5_1'] = build_net('conv',net['pool4'],get_weight_bias(vgg_layers,28))
    net['conv5_2'] = build_net('conv',net['conv5_1'],get_weight_bias(vgg_layers,30))
    net['conv5_3'] = build_net('conv',net['conv5_2'],get_weight_bias(vgg_layers,32))
    net['conv5_4'] = build_net('conv',net['conv5_3'],get_weight_bias(vgg_layers,34))
    net['pool5']   = build_net('pool',net['conv5_4'])
    return net

In [8]:
def build_content_loss(p, x):
    """
    コンテンツと出力の誤差
    """
    M = p.shape[1]*p.shape[2]
    N = p.shape[3]
    loss = (1./(2* N**0.5 * M**0.5 )) * tf.reduce_sum(tf.pow((x - p),2))  
    return loss

In [9]:
def gram_matrix(x, area, depth):
    x1 = tf.reshape(x,(area,depth))
    g = tf.matmul(tf.transpose(x1), x1)
    return g

In [10]:
def gram_matrix_val(x, area, depth):
    x1 = x.reshape(area,depth)
    g = np.dot(x1.T, x1)
    return g

In [11]:
def build_style_loss(a, x):
    M = a.shape[1]*a.shape[2]
    N = a.shape[3]
    A = gram_matrix_val(a, M, N )
    G = gram_matrix(x, M, N )
    loss = (1./(4 * N**2 * M**2)) * tf.reduce_sum(tf.pow((G - A),2))
    return loss

In [12]:
def read_image(path):
    """
    画像を読み込む関数
    """
    image = scipy.misc.imread(path)
    image = scipy.misc.imresize(image,(IMAGE_H,IMAGE_W))
    image = image[np.newaxis,:,:,:] 
    image = image - MEAN_VALUES
    return image

In [13]:
def write_image(path, image):
    """
    生成された画像を保存する関数
    """
    image = image + MEAN_VALUES
    image = image[0]
    image = np.clip(image, 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)

In [14]:
# VGG19モデルの作成
net = build_vgg19(VGG_MODEL)
# ホワイトノイズ
noise_img = np.random.uniform(-20, 20, (1, IMAGE_H, IMAGE_W, 3)).astype('float32')
# 画像の読み込み
content_img = read_image(CONTENT_IMG)
style_img = read_image(STYLE_IMG)

In [15]:
# 初期化
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [16]:
"""
画風変換の出力を調節するにはここを変更
"""
# 各種パラメータの設定
INI_NOISE_RATIO = 0.7 # ホワイトノイズの重み
STYLE_STRENGTH = 500 # スタイルの強さ
ITERATION = 5000 # 最適化回数

# コンテンツ画像と出力画像で誤差を取る層
CONTENT_LAYERS =[('conv4_2',1.)]
# スタイル画像と出力画像で誤差を取る層
STYLE_LAYERS=[('conv1_1',1.),('conv2_1',1.),('conv3_1',1.),('conv4_1',1.),('conv5_1',1.)]

In [17]:
sess.run([net['input'].assign(content_img)])
cost_content = sum(map(lambda l,: l[1]*build_content_loss(sess.run(net[l[0]]) ,  net[l[0]]), CONTENT_LAYERS))

In [18]:
sess.run([net['input'].assign(style_img)])
cost_style = sum(map(lambda l: l[1]*build_style_loss(sess.run(net[l[0]]) ,  net[l[0]]), STYLE_LAYERS))

In [19]:
cost_total = cost_content + STYLE_STRENGTH * cost_style
optimizer = tf.train.AdamOptimizer(2.0)

In [20]:
train = optimizer.minimize(cost_total)
sess.run( tf.global_variables_initializer())
sess.run(net['input'].assign( INI_NOISE_RATIO* noise_img + (1.-INI_NOISE_RATIO) * content_img))

array([[[[ -8.90594292,   4.98616314,  23.42633438],
         [  2.85777044,  21.7790432 ,  37.6257782 ],
         [  7.12507439,   2.30034161,  16.07798004],
         ..., 
         [ 15.64326286,  20.30531311,  35.65568161],
         [ 14.09522915,   6.40278625,  41.16028595],
         [  5.11479378,  18.35180473,  28.48310089]],

        [[ -1.13934422,  23.13929176,  39.82580185],
         [ 12.12512302,   9.65085411,  18.56409264],
         [  0.45161247,   4.92058039,  22.86688042],
         ..., 
         [ 12.19301987,  22.99743271,  18.88684082],
         [ 11.84660912,  27.66487694,  31.53080559],
         [  1.45625234,   7.65340614,  36.30850601]],

        [[ -6.69076014,   7.09409809,  33.79944229],
         [ -3.20938063,  12.97106647,  39.39369202],
         [ -0.87970066,  -2.3080008 ,  24.8520546 ],
         ..., 
         [ 12.11437702,  11.57104301,  41.77233887],
         [ 16.6821022 ,  15.9099493 ,  20.82740211],
         [  6.0095911 ,  28.42125511,  24.60186005

In [21]:
# 保存先ディレクトリが存在しないときは作成
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [22]:
for i in range(ITERATION):
    sess.run(train)
    # 100回ごとに経過を表示、画像を保存
    if i%100 ==0:
        result_img = sess.run(net['input'])
        print ("ITERATION: ",i,", ",sess.run(cost_total))
        write_image(os.path.join(OUTPUT_DIR,'%s.png'%(str(i).zfill(4))),result_img)

ITERATION:  0 ,  5.3408e+12
ITERATION:  100 ,  6.16613e+10
ITERATION:  200 ,  2.52467e+10
ITERATION:  300 ,  1.47362e+10
ITERATION:  400 ,  1.01176e+10
ITERATION:  500 ,  7.45694e+09
ITERATION:  600 ,  5.70736e+09
ITERATION:  700 ,  4.48629e+09
ITERATION:  800 ,  3.60164e+09
ITERATION:  900 ,  2.94134e+09
ITERATION:  1000 ,  2.43768e+09
ITERATION:  1100 ,  2.0538e+09
ITERATION:  1200 ,  1.75485e+09
ITERATION:  1300 ,  1.5175e+09
ITERATION:  1400 ,  2.71715e+09
ITERATION:  1500 ,  2.33854e+09
ITERATION:  1600 ,  1.17687e+09
ITERATION:  1700 ,  9.64381e+08
ITERATION:  1800 ,  8.91689e+08
ITERATION:  1900 ,  8.97573e+08
ITERATION:  2000 ,  7.71974e+08
ITERATION:  2100 ,  7.26161e+08
ITERATION:  2200 ,  8.8141e+08
ITERATION:  2300 ,  7.11507e+08
ITERATION:  2400 ,  5.99756e+08
ITERATION:  2500 ,  9.85718e+08
ITERATION:  2600 ,  5.67326e+08
ITERATION:  2700 ,  6.24285e+08
ITERATION:  2800 ,  5.5902e+08
ITERATION:  2900 ,  4.83905e+08
ITERATION:  3000 ,  5.14623e+09
ITERATION:  3100 ,  4.812

In [23]:
write_image(os.path.join(OUTPUT_DIR,OUTPUT_IMG),result_img)